### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 3.5.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run priors_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    prob = params[3:6]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2,.3,2,.5, .8, .2])

### New function to sim data for multiple subjects

In [9]:
def sim_mult_subject_data(n, tn, params):
    data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta', 'iProbA', 'iProbB', 'iProbC'])
    for x in range(1, n+1):
        sdt = sim_data(tn, params)
        sdt = pd.DataFrame.from_dict(sdt)
        sdt['Subject'] = [x for t in range(0,tn*3)]
        sdt['Alpha_gain'] = params[0]
        sdt['Alpha_loss'] = params[1]
        sdt['Beta'] = params[2]
        sdt['iProbA'] = params[3]
        sdt['iProbB'] = params[4]
        sdt['iProbC'] = params[5]
        data = pd.concat([data,sdt])
    return data

In [10]:
#data = sim_mult_subject_data(10, 15, [.2,.3, 2, .5, .8, .2])
#data

### Create param space

In [11]:
# create parameter space
alpha_gains = [round(x * 0.25,1) for x in range(1,5)]
alpha_loses = [round(x * 0.25,1) for x in range(1,5)]
betas = [x * 4 for x in range(1,5)]
iprobA = [round(x * .25,1) for x in range(1,4)]
iprobB = [round(x * .25,1) for x in range(1,4)]
iprobC = [round(x * .25,1) for x in range(1,4)]

combos = list(itertools.product(alpha_gains, alpha_loses, betas, iprobA, iprobB, iprobC))
len(combos)
#for r in combos: print(r)

1728

### Simulate data

In [12]:
data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 
                              'Alpha_gain', 'Alpha_loss','Beta', 'iProbA', 'iProbB', 'iProbC'])
for p in range(len(combos)):
    dt = sim_mult_subject_data(1, 15, combos[p])
    dt['Subject'] = dt['Subject'] + p
    data = pd.concat([data, dt])
data

data.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_with_priors_model_data.csv', index = False)

### New function to fit model to multiple subjects

In [14]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'a_gain', 'a_loss', 'beta', 'iProbA', 'iProbB', 'iProbC'])
    a = pd.unique(data['Subject'])
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==sub]
        dt = df.to_dict()
        params = model_fit(dt)
        #print('Params: ', params)
        line = {'Subject': sub, 'a_gain': params.x[0], 'a_loss': params.x[1], 'beta':params.x[2], 
                'iProbA':params.x[3], 'iProbB':params.x[4], 'iProbC':params.x[5]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [15]:
precover = model_fit_mult_subject(data)
precover.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_with_priors_model_fit.csv', index = False)

Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9
Subject:  10
Subject:  11
Subject:  12
Subject:  13
Subject:  14
Subject:  15
Subject:  16
Subject:  17
Subject:  18
Subject:  19
Subject:  20
Subject:  21
Subject:  22
Subject:  23
Subject:  24
Subject:  25
Subject:  26
Subject:  27
Subject:  28
Subject:  29
Subject:  30
Subject:  31
Subject:  32
Subject:  33
Subject:  34
Subject:  35
Subject:  36
Subject:  37
Subject:  38
Subject:  39
Subject:  40
Subject:  41
Subject:  42
Subject:  43
Subject:  44
Subject:  45
Subject:  46
Subject:  47
Subject:  48
Subject:  49
Subject:  50
Subject:  51
Subject:  52
Subject:  53
Subject:  54
Subject:  55
Subject:  56
Subject:  57
Subject:  58
Subject:  59
Subject:  60
Subject:  61
Subject:  62
Subject:  63
Subject:  64
Subject:  65
Subject:  66
Subject:  67
Subject:  68
Subject:  69
Subject:  70
Subject:  71
Subject:  72
Subject:  73
Subject:  74
Subject:  75
Subject:  76
Subject:  77
Subject:

Subject:  594
Subject:  595
Subject:  596
Subject:  597
Subject:  598
Subject:  599
Subject:  600
Subject:  601
Subject:  602
Subject:  603
Subject:  604
Subject:  605
Subject:  606
Subject:  607
Subject:  608
Subject:  609
Subject:  610
Subject:  611
Subject:  612
Subject:  613
Subject:  614
Subject:  615
Subject:  616
Subject:  617
Subject:  618
Subject:  619
Subject:  620
Subject:  621
Subject:  622
Subject:  623
Subject:  624
Subject:  625
Subject:  626
Subject:  627
Subject:  628
Subject:  629
Subject:  630
Subject:  631
Subject:  632
Subject:  633
Subject:  634
Subject:  635
Subject:  636
Subject:  637
Subject:  638
Subject:  639
Subject:  640
Subject:  641
Subject:  642
Subject:  643
Subject:  644
Subject:  645
Subject:  646
Subject:  647
Subject:  648
Subject:  649
Subject:  650
Subject:  651
Subject:  652
Subject:  653
Subject:  654
Subject:  655
Subject:  656
Subject:  657
Subject:  658
Subject:  659
Subject:  660
Subject:  661
Subject:  662
Subject:  663
Subject:  664
Subjec

Subject:  1168
Subject:  1169
Subject:  1170
Subject:  1171
Subject:  1172
Subject:  1173
Subject:  1174
Subject:  1175
Subject:  1176
Subject:  1177
Subject:  1178
Subject:  1179
Subject:  1180
Subject:  1181
Subject:  1182
Subject:  1183
Subject:  1184
Subject:  1185
Subject:  1186
Subject:  1187
Subject:  1188
Subject:  1189
Subject:  1190
Subject:  1191
Subject:  1192
Subject:  1193
Subject:  1194
Subject:  1195
Subject:  1196
Subject:  1197
Subject:  1198
Subject:  1199
Subject:  1200
Subject:  1201
Subject:  1202
Subject:  1203
Subject:  1204
Subject:  1205
Subject:  1206
Subject:  1207
Subject:  1208
Subject:  1209
Subject:  1210
Subject:  1211
Subject:  1212
Subject:  1213
Subject:  1214
Subject:  1215
Subject:  1216
Subject:  1217
Subject:  1218
Subject:  1219
Subject:  1220
Subject:  1221
Subject:  1222
Subject:  1223
Subject:  1224
Subject:  1225
Subject:  1226
Subject:  1227
Subject:  1228
Subject:  1229
Subject:  1230
Subject:  1231
Subject:  1232
Subject:  1233
Subject:  

Subject:  1715
Subject:  1716
Subject:  1717
Subject:  1718
Subject:  1719
Subject:  1720
Subject:  1721
Subject:  1722
Subject:  1723
Subject:  1724
Subject:  1725
Subject:  1726
Subject:  1727
Subject:  1728


In [16]:
print(precover)

      Subject    a_gain    a_loss      beta    iProbA    iProbB        iProbC
0         1.0  0.183672  0.377251  0.530094  0.748144  0.315278  2.370493e-01
1         2.0  0.025692  0.219316  0.822204  0.348078  0.920549  1.868144e-01
2         3.0  0.047917  0.103054  0.610731  0.635498  0.808001  2.493581e-01
3         4.0  0.295093  0.313911  0.204913  0.905351  0.881359  7.610378e-02
4         5.0  0.558135  0.801398  0.405915  0.999549  0.031381  3.117479e-02
5         6.0  0.216236  0.635614  0.584403  0.901659  0.943759  1.110729e-01
6         7.0  0.119587  0.793985  0.396019  0.907849  0.922469  1.187756e-03
7         8.0  0.145555  0.493833  0.284926  0.821866  0.165471  4.843744e-02
8         9.0  0.695584  0.349981  0.916239  0.999999  0.990071  2.204892e-01
9        10.0  0.536430  0.673209  0.540767  0.998636  0.069653  1.046305e-01
10       11.0  0.881626  0.282804  0.997073  1.000000  0.294958  3.331895e-01
11       12.0  0.721825  0.417176  0.530264  0.967274  0.275787 